In [2]:
import pandas as pd
import numpy as np
import sys 
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
import itertools
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

from joblib import Parallel, delayed, dump, load
# sys.path.insert(0, '../DevCode')


pd.set_option('display.expand_frame_repr', False)
pd.options.display.max_rows = 500
sys.path.append('../code')
import pickle

<h2> Read from our Tuple Data </h2>

In [3]:
valid = pd.read_pickle('../code/data/valid_tuple.pkl')
train = pd.read_pickle('../code/data/train_tuple.pkl')


In [ ]:
train[0][0]

array([1., 2., 3., 4., 5., 4., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [48]:
train[0][:,]

array([[1.00000e+00, 2.00000e+00, 3.00000e+00, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [6.00000e+00, 4.00000e+00, 5.00000e+00, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [1.10000e+01, 4.00000e+00, 1.20000e+01, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       ...,
       [2.39386e+05, 4.00000e+00, 1.19000e+02, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [3.46000e+02, 2.00000e+00, 8.16300e+03, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [3.46000e+02, 4.00000e+00, 5.00000e+00, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00]])

In [153]:
defaults_new = {
    'boosting_type':['rf'],
    'n_estimators':[75],
    'max_depth':[8,13, 21, 34],
    'num_leaves':[34**2],
    'min_child_samples':[10,25,100],
    'subsample':[0.98],
    'colsample_bytree': [0.7],
    'reg_alpha':[0.1, 0.3],
    'reg_lambda':[0.1, 0.3],
    'random_state': [5],
    'verbosity':[-1],
    'n_jobs':[1],
    'subsample_freq':[1],
    'learning_rate':[1.0],
    'class_weight':[{0:0.1, 1:0.3, 2:0.3, 3:0.3}]
    
}

In [4]:
defaults_new = {
    'boosting_type':['rf'],
    'n_estimators':[150],
    'max_depth':[8,13, 21, 34],
    'num_leaves':[34**2],
    'min_child_samples':[10,25,100],
    'subsample':[0.98],
    'colsample_bytree': [0.7],
    'reg_alpha':[0.1, 0.3],
    'reg_lambda':[0.1, 0.3],
    'random_state': [5],
    'verbosity':[-1],
    'n_jobs':[2],
    'subsample_freq':[1],
    'learning_rate':[1.0],
    'class_weight':[{0:0.01, 1:0.35, 2:0.35, 3:0.35}]
    
}

In [59]:
defaults_new = {
    'boosting_type':['gbdt'],
    # 'n_estimators':[250],
    'n_estimators':[250],
    'max_depth':[13],
    'num_leaves':[13 **2],
    'min_child_samples':[25],
    'subsample':[0.99],
    'subsample_freq':[1],
    'colsample_bytree': [0.8],
    # 'reg_alpha':[0.1],
    # 'reg_lambda':[0.3],
    'random_state': [5],
    'verbosity':[-1],
    'n_jobs':[2],
    'learning_rate':[0.01, 0.1, 0.3],
    # 'class_weight':[{0:0.25, 1:0.3, 2:0.3, 3:0.15}]
    'class_weight':[{0:0.01, 1:0.65, 2:0.25, 3:0.10}]
    
}

In [60]:
combinations = itertools.product(*defaults_new.values())

sub_dicts = [dict(zip(defaults_new.keys(), combo)) for combo in combinations]

hypersearch_list = [LGBMClassifier(**params) for params in sub_dicts]

In [105]:
from sklearn.ensemble import AdaBoostClassifier 
from sklearn.tree import DecisionTreeClassifier
import itertools

defaults_new = {
    'n_estimators': [50],  # Adjust the number of estimators as needed
    'learning_rate': [0.01, 0.1],
    'base_estimator': [DecisionTreeClassifier(max_depth=8), DecisionTreeClassifier(max_depth=21)],
    # 'base_estimator': [base_model],
    'random_state': [5],
}

# Create combinations of all possible parameters
combinations = itertools.product(*defaults_new.values())

# Combine the combinations with their values
sub_dicts = [dict(zip(defaults_new.keys(), combo)) for combo in combinations]

# Create a list of models to plug into the Parallel Joblib function
hypersearch_list = [AdaBoostClassifier(**params) for params in sub_dicts]

In [84]:
# defaults_new = {
#     'n_estimators':[100, 150],
#     'max_depth':[3,4,5,6,7],
#     'subsample':[0.98],
#     'colsample_bytree': [0.1,0.3],
#     'colsample_bynode':[0.1,0.3],
#     'learning_rate': [0.1, 0.3, 0.01],
#     'random_state': [5],
#     'n_jobs':[2]
# }

# #create combinations of all possible parameters
# combinations = itertools.product(*defaults_new.values())

# #combine the combinations with their values
# sub_dicts = [dict(zip(defaults_new.keys(), combo)) for combo in combinations]

# #create a list of models to plug into the Parallel Joblib function
# hypersearch_list = [XGBClassifier(**params) for params in sub_dicts]

In [129]:
len(hypersearch_list)

320

<h2> Split our Data into the Training and Validation Data </h2>

In [61]:
X_train, y_train = train[0], train[1]
X_test, y_test = valid[0], valid[1]

In [47]:
X_train.shape

(440619, 47)

In [48]:
X_train[0]

array([1., 2., 3., 4., 5., 4., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [49]:
y_train.shape

(440619,)

In [50]:
X_test

array([[1.70000e+01, 1.80000e+01, 7.00000e+00, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [4.46852e+05, 4.00000e+00, 5.00000e+00, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [4.57000e+02, 4.00000e+00, 5.00000e+00, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       ...,
       [1.70000e+01, 1.80000e+01, 7.00000e+00, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [1.70000e+01, 1.80000e+01, 7.00000e+00, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [1.70000e+01, 1.80000e+01, 7.00000e+00, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00]])

<h2> Create a training method that will run each model. <br> 
Then the function append the scores for each fold and create a tuple of these values </h2>

In [62]:
def training(input_model):
    model = input_model
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    score = accuracy_score(y_test, y_pred)


    data_tuple = (score,  model.get_params(), model)

    return data_tuple

<h2> This method utilizes the Parallel library to train multiple models on multiple cores </h2>

In [63]:
def run_multi_core():
    results = Parallel(n_jobs=22)(delayed(training)(i) for i in hypersearch_list)
    return results

<h2> Creates a final dataframe based on the results obtained from the run_mulit_core function </h2>

In [64]:
def create_df():
    data_tuple = run_multi_core()
    data = pd.DataFrame(data_tuple,
                        columns=['total_score', 'params', 'model'])
    data = data.sort_values(by='total_score', ascending=False)
    return data

In [65]:
results = create_df()

In [66]:
results.head(10)

,total_score,params,model
0,0.793069,"{'boosting_type': 'gbdt', 'class_weight': {0: ...","LGBMClassifier(class_weight={0: 0.01, 1: 0.65,..."
1,0.787170,"{'boosting_type': 'gbdt', 'class_weight': {0: ...","LGBMClassifier(class_weight={0: 0.01, 1: 0.65,..."
2,0.253204,"{'boosting_type': 'gbdt', 'class_weight': {0: ...","LGBMClassifier(class_weight={0: 0.01, 1: 0.65,..."


In [161]:
results.head(10)

,total_score,params,model
39,0.794552,"{'boosting_type': 'rf', 'class_weight': {0: 0....","LGBMClassifier(boosting_type='rf',\n ..."
27,0.794234,"{'boosting_type': 'rf', 'class_weight': {0: 0....","LGBMClassifier(boosting_type='rf',\n ..."
37,0.793863,"{'boosting_type': 'rf', 'class_weight': {0: 0....","LGBMClassifier(boosting_type='rf',\n ..."
38,0.793429,"{'boosting_type': 'rf', 'class_weight': {0: 0....","LGBMClassifier(boosting_type='rf',\n ..."
25,0.793260,"{'boosting_type': 'rf', 'class_weight': {0: 0....","LGBMClassifier(boosting_type='rf',\n ..."
26,0.792815,"{'boosting_type': 'rf', 'class_weight': {0: 0....","LGBMClassifier(boosting_type='rf',\n ..."
36,0.792656,"{'boosting_type': 'rf', 'class_weight': {0: 0....","LGBMClassifier(boosting_type='rf',\n ..."
24,0.792063,"{'boosting_type': 'rf', 'class_weight': {0: 0....","LGBMClassifier(boosting_type='rf',\n ..."
43,0.790114,"{'boosting_type': 'rf', 'class_weight': {0: 0....","LGBMClassifier(boosting_type='rf',\n ..."
31,0.789860,"{'boosting_type': 'rf', 'class_weight': {0: 0....","LGBMClassifier(boosting_type='rf',\n ..."


In [57]:
for i in range(3):
    print(results.iloc[i].params)

{'boosting_type': 'gbdt', 'class_weight': {0: 0.01, 1: 0.55, 2: 0.35, 3: 0.1}, 'colsample_bytree': 0.8, 'importance_type': 'split', 'learning_rate': 0.01, 'max_depth': 13, 'min_child_samples': 25, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 250, 'n_jobs': 2, 'num_leaves': 169, 'objective': None, 'random_state': 5, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'silent': 'warn', 'subsample': 0.99, 'subsample_for_bin': 200000, 'subsample_freq': 1, 'verbosity': -1}
{'boosting_type': 'gbdt', 'class_weight': {0: 0.01, 1: 0.55, 2: 0.35, 3: 0.1}, 'colsample_bytree': 0.8, 'importance_type': 'split', 'learning_rate': 0.1, 'max_depth': 13, 'min_child_samples': 25, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 250, 'n_jobs': 2, 'num_leaves': 169, 'objective': None, 'random_state': 5, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'silent': 'warn', 'subsample': 0.99, 'subsample_for_bin': 200000, 'subsample_freq': 1, 'verbosity': -1}
{'boosting_type': 'gbdt', 'class_weight': {

In [25]:
import plotly.express as px

In [26]:
test_model = results.iloc[0].model
# test_model = r.iloc[0].model
importances = test_model.feature_importances_

In [117]:
importances

array([1.90156385e-02, 1.83054535e-02, 1.15067745e-04, 5.00527959e-03,
       1.51960386e-01, 1.57798803e-02, 9.20609312e-02, 2.96231010e-02,
       7.39764734e-02, 2.76702867e-02, 6.89769710e-02, 1.62568129e-02,
       5.32629063e-02, 2.23914986e-02, 6.01130308e-02, 5.09845979e-02,
       3.90125921e-02, 3.48167130e-02, 4.02578688e-02, 2.92617735e-02,
       3.42149171e-02, 1.18992934e-02, 2.85921363e-02, 1.04954444e-02,
       2.52177284e-02, 3.66939898e-03, 8.84396418e-03, 2.05280791e-03,
       1.58225771e-02, 2.16120617e-03, 3.75424994e-03, 1.01647684e-03,
       5.13393420e-04, 4.31040998e-04, 1.05521002e-03, 5.30934451e-04,
       2.45927714e-04, 1.71059001e-05, 2.45261494e-05, 2.23121833e-04,
       6.66131242e-05, 3.40830504e-05, 2.67756252e-05, 1.45737797e-04,
       9.80320547e-05, 0.00000000e+00, 3.43209466e-08])

In [140]:
test_model.n_features_in_

47

In [141]:
dummy_cols = [f'index_{i}' for i in range(1, 48)]

In [125]:
len(dummy_cols)

47

In [142]:
test_model.feature_name_

['Column_0',
 'Column_1',
 'Column_2',
 'Column_3',
 'Column_4',
 'Column_5',
 'Column_6',
 'Column_7',
 'Column_8',
 'Column_9',
 'Column_10',
 'Column_11',
 'Column_12',
 'Column_13',
 'Column_14',
 'Column_15',
 'Column_16',
 'Column_17',
 'Column_18',
 'Column_19',
 'Column_20',
 'Column_21',
 'Column_22',
 'Column_23',
 'Column_24',
 'Column_25',
 'Column_26',
 'Column_27',
 'Column_28',
 'Column_29',
 'Column_30',
 'Column_31',
 'Column_32',
 'Column_33',
 'Column_34',
 'Column_35',
 'Column_36',
 'Column_37',
 'Column_38',
 'Column_39',
 'Column_40',
 'Column_41',
 'Column_42',
 'Column_43',
 'Column_44',
 'Column_45',
 'Column_46']

In [143]:
test_model = results.iloc[0].model
# test_model = r.iloc[0].model
importances = test_model.feature_importances_
# features = test_model.feature_name_
features = dummy_cols

plot_df = pd.DataFrame({'Features':features, 'Importances':importances})
plot_df = plot_df.sort_values(by='Importances', ascending=True)


fig = px.bar(plot_df, x='Importances', y='Features', template='plotly_dark',width=1200, height=700)
fig

In [27]:
temporary_model = results.iloc[0].model

In [67]:
dump(results.iloc[0].model, '../code/data/gb_model.joblib')

['../code/data/gb_model.joblib']

In [88]:
dump(results.iloc[0].model, 'gb_model.joblib')

['gb_model.joblib']

In [21]:

temporary_model.fit(X_train, y_train)
y_pred = temporary_model.predict(X_test)

In [22]:
y_test

array([2, 1, 0, ..., 3, 3, 3])

In [23]:
y_pred

array([2, 1, 0, ..., 2, 2, 2])